## Download all the data we need for Wat zei je benchmarks

Training
- news-commentary-v13.de-en
- wmt13-commoncrawl.de-en
- wmt13-europarl.de-en

Test
- wmt14-newstest2014-de-en

Test OOD
- news-commentary-v14.nl-en

In [9]:
training_sets = [
    {
        "link": "http://data.statmt.org/wmt18/translation-task/training-parallel-nc-v13.tgz",
        "subfolder": "",
        "de_file": "news-commentary-v13.de-en.en",
        "en_file": "news-commentary-v13.de-en.en"
    },
    {
        "link": "https://www.statmt.org/wmt13/training-parallel-commoncrawl.tgz",
        "subfolder": "",
        "de_file": "commoncrawl.de-en.de",
        "en_file": "commoncrawl.de-en.en"
    },
    {
        "link": "https://www.statmt.org/wmt13/training-parallel-europarl-v7.tgz",
        "subfolder": "training/",
        "de_file": "europarl-v7.de-en.de",
        "en_file": "europarl-v7.de-en.en"
    }
]
test_set = {
    "link":  "https://www.statmt.org/wmt14/test.tgz",
    "subfolder": "test/",
    "de_file": "newstest2014-deen-src.de.sgm",
    "en_file": "newstest2014-deen-src.en.sgm"
}

test_ood = {
    "link": "https://data.statmt.org/news-commentary/v14/training/news-commentary-v14.es-nl.tsv.gz",
    "subfolder": ""
}


In [ ]:
import requests
import tarfile
import gzip
import shutil
import os

data_directory = "./local/data"
def download_and_extract(url, de_file=None, en_file=None):
    local_filename = url.split('/')[-1]
    with requests.get(url, stream=True) as r:
        r.raise_for_status()
        with open(local_filename, 'wb') as f:
            for chunk in r.iter_content(chunk_size=8192):
                f.write(chunk)
    if local_filename.endswith(".tgz") or local_filename.endswith(".tar.gz"):
        with tarfile.open(local_filename, "r:gz") as tar:
            if de_file is None or en_file is None:
                tar.extractall(path=data_directory)
            else:
                for member in tar.getmembers():
                    if member.name.endswith(de_file) or member.name.endswith(en_file):
                        tar.extract(member, path=data_directory)

    if local_filename.endswith(".gz") and not local_filename.endswith(".tar.gz"):
        with gzip.open(local_filename, 'rb') as f_in:
            with open(os.path.join(data_directory, local_filename[:-3]), 'wb') as f_out:
                shutil.copyfileobj(f_in, f_out)
    os.remove(local_filename)

# Download and extract training sets
for dataset in training_sets:
    download_and_extract(dataset["link"], dataset["de_file"], dataset["en_file"])

# Download and extract test set
download_and_extract(test_set["link"], test_set["de_file"], test_set["en_file"])

# Download and extract ood test set
download_and_extract(test_ood["link"])